## Credit Score Project

In [1]:
#importing necessary libaries 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, precision_score, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Importing the dataset for train
train_dataset = pd.read_csv('/workspaces/codespaces-blank/credit_score_project/data/data_devsample.csv')
train_dataset.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,MEAN_AMTCR_OVERDUE_0M_INFM_TYPE_EQ_ACTIVE_DIV_MEAN_AMTCR_0M_INFM_TYPE_EQ_ACTIVE,MEAN_AMTCR_OVERDUE_0M_12M_TYPE_EQ_ACTIVE_DIV_MEAN_AMTCR_0M_12M_TYPE_EQ_ACTIVE,MEAN_AMTCR_OVERDUE_0M_INFM_TYPE_EQ_CLOSED_DIV_MEAN_AMTCR_0M_INFM_TYPE_EQ_CLOSED,MEAN_AMTCR_OVERDUE_0M_12M_TYPE_EQ_CLOSED_DIV_MEAN_AMTCR_0M_12M_TYPE_EQ_CLOSED,FIRST_TRANSACTION_TIME_MONTHS,LAST_TRANSACTION_TIME_MONTHS,TIME,BASE,DAY,MONTH
0,215138,0.0,Cash loans,M,N,Y,0.0,135000.0,646920.0,25195.5,...,0.0,NaN,0.0,NaN,37.0,28.0,2018-06-23,1,20180623,201806
1,325963,0.0,Cash loans,M,Y,Y,0.0,135000.0,740218.5,40284.0,...,0.0,0.0,0.0,NaN,3.0,18.0,2018-06-16,1,20180616,201806
2,206923,0.0,Cash loans,F,N,Y,0.0,180000.0,339241.5,12919.5,...,0.0,NaN,0.0,NaN,73.0,61.0,2018-06-19,1,20180619,201806
3,265855,0.0,Cash loans,F,N,Y,0.0,180000.0,1264428.0,37098.0,...,0.0,0.0,NaN,NaN,36.0,4.0,2018-08-09,1,20180809,201808
4,359795,0.0,Revolving loans,F,N,Y,NaN,76500.0,202500.0,10125.0,...,NaN,NaN,0.0,0.0,12.0,12.0,2018-11-13,1,20181113,201811


In [42]:
first_bacth = train_dataset.iloc[:, :50]
second_batch = train_dataset.iloc[:, 50:]

In [46]:
second_batch.isnull().sum()

ENTRANCES_AVG                   40256
FLOORSMAX_AVG                   39779
FLOORSMIN_AVG                   54406
LANDAREA_AVG                    47531
LIVINGAPARTMENTS_AVG            54775
                                ...  
LAST_TRANSACTION_TIME_MONTHS    30353
TIME                                0
BASE                                0
DAY                                 0
MONTH                               0
Length: 147, dtype: int64

In [13]:
train_dataset.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,MEAN_AMTCR_OVERDUE_0M_12M_DIV_MEAN_AMTCR_0M_12M,MEAN_AMTCR_OVERDUE_0M_INFM_TYPE_EQ_ACTIVE_DIV_MEAN_AMTCR_0M_INFM_TYPE_EQ_ACTIVE,MEAN_AMTCR_OVERDUE_0M_12M_TYPE_EQ_ACTIVE_DIV_MEAN_AMTCR_0M_12M_TYPE_EQ_ACTIVE,MEAN_AMTCR_OVERDUE_0M_INFM_TYPE_EQ_CLOSED_DIV_MEAN_AMTCR_0M_INFM_TYPE_EQ_CLOSED,MEAN_AMTCR_OVERDUE_0M_12M_TYPE_EQ_CLOSED_DIV_MEAN_AMTCR_0M_12M_TYPE_EQ_CLOSED,FIRST_TRANSACTION_TIME_MONTHS,LAST_TRANSACTION_TIME_MONTHS,BASE,DAY,MONTH
count,80000.000000,80000.000000,55035.000000,8.000000e+04,8.000000e+04,79994.000000,7.993700e+04,80000.000000,59780.00000,80000.000000,...,16914.0,28407.0,14243.0,3.715600e+04,3833.0,49647.000000,49647.000000,80000.0,8.000000e+04,80000.000000
mean,278160.918850,0.080200,0.324648,1.682663e+05,5.982419e+05,27115.320924,5.376062e+05,0.020899,-16031.07740,63636.234238,...,inf,inf,inf,6.318908e-07,0.0,36.875904,36.835841,1.0,2.018067e+07,201806.523287
std,102712.983646,0.271604,0.637932,1.092775e+05,4.002051e+05,14471.545555,3.674006e+05,0.013846,4365.52899,141122.463859,...,NaN,NaN,NaN,7.486569e-05,0.0,26.356657,26.348293,0.0,3.459396e+02,3.457120
min,100002.000000,0.000000,0.000000,2.610000e+04,4.500000e+04,1980.000000,4.050000e+04,0.000533,-25187.00000,-17531.000000,...,0.0,0.0,0.0,0.000000e+00,0.0,1.000000,1.000000,1.0,2.018010e+07,201801.000000
25%,189175.750000,0.000000,0.000000,1.125000e+05,2.700000e+05,16573.500000,2.385000e+05,0.010006,-19690.25000,-2747.000000,...,0.0,0.0,0.0,0.000000e+00,0.0,15.000000,15.000000,1.0,2.018040e+07,201804.000000
50%,278856.500000,0.000000,0.000000,1.485000e+05,5.135310e+05,24939.000000,4.500000e+05,0.018850,-15754.00000,-1215.000000,...,0.0,0.0,0.0,0.000000e+00,0.0,31.000000,31.000000,1.0,2.018070e+07,201807.000000
75%,366960.000000,0.000000,0.000000,2.025000e+05,8.086500e+05,34644.375000,6.795000e+05,0.028663,-12394.00000,-291.000000,...,0.0,0.0,0.0,0.000000e+00,0.0,54.000000,54.000000,1.0,2.018100e+07,201810.000000
max,456251.000000,1.000000,12.000000,1.350000e+07,4.050000e+06,225000.000000,4.050000e+06,0.072508,-7489.00000,365243.000000,...,inf,inf,inf,1.284610e-02,0.0,96.000000,96.000000,1.0,2.018123e+07,201812.000000
